### Import library
Nous allons importer les bibliothèque numpy et pandas qui sont necessaire pour le traitement.

In [71]:
import pandas as pd
import numpy as np

### Import Data
Nous allons importer les données depuis le répertoire RAwData pour enfin les traiter et transformer

In [72]:
n=['TimesTamp', 'ECP', 'Antenna', 'RSSI', 'Channel', 'Adress']
p='../1-RawData/P30.csv'
d=','
def importData(path, delimit,cols):
    return pd.read_csv(path, sep=delimit,names=cols)

data=importData(p,d,n)
data = data.drop([0],axis=0) #Suppression de la 1ere ligne

data.head(10)

TimesTamp                       ECP Antenna   RSSI Channel  \
1   1581079415482429  30000000000000000001785C       1    -55   865.7   
2   1581079415484133  300000000000000000017857       1  -60.5   865.7   
3   1581079415486862              245390003866       1    -67   865.7   
4   1581079415507659  30000000000000000001785C       1    -55   865.7   
5   1581079415508791  300000000000000000017857       1  -60.5   865.7   
6   1581079415511981              245390003866       1    -67   865.7   
7   1581079415543721  300000000000000000017857       2  -59.5   865.7   
8   1581079415546450  300000000000000000017885       2    -52   865.7   
9   1581079415548796              245390003868       2  -68.5   865.7   
10  1581079415550258  30000000000000000001785C       2    -52   865.7   

          Adress  
1   192.168.0.10  
2   192.168.0.10  
3   192.168.0.10  
4   192.168.0.10  
5   192.168.0.10  
6   192.168.0.10  
7   192.168.0.10  
8   192.168.0.10  
9   192.168.0.10  
10  192.168.0.10

### TYPAGE DES CHAMPS
Nos allons typer les différentes colonnes du dataSet avec les types primaires de python 

In [73]:
def typage(data):
    data['ECP']=data['ECP'].astype(str)
    data['TimesTamp']=data['TimesTamp'].astype('int64')
    data['RSSI']=data['RSSI'].astype('float64')
    data['Antenna']=data['Antenna'].astype('int64')
    return data

data=typage(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7122 entries, 1 to 7122
Data columns (total 6 columns):
TimesTamp    7122 non-null int64
ECP          7122 non-null object
Antenna      7122 non-null int64
RSSI         7122 non-null float64
Channel      7122 non-null object
Adress       7122 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 389.5+ KB


### False Positif
cette section est exécuter que lors de la génération des données de d'apprentissage et de test. Connaissant les données fausses positives, nous allons définir une colonne pour toutes les données qui sera 1 pour les parasites et 1 pour les autres. 

In [54]:
#La liste de nos étiquettes falsePositive
fp_list=['245390003868','245390003869','300000000000000000017882','300000000000000000017884','300000000000000000017851',
         '300000000000000000017856','300000000000000000017857','300000000000000000017858','300000000000000000017860',
         '30000000000000000001785A','30000000000000000001785C','3000000000000000000178BC',
         '30000000000000000001785D','30000000000000000001785F']

def setFalsePositive(fp_list, dataSet):
    fp={}
    for e in dataSet.ECP:
        fp[e]=0
        for f in fp_list:
            if(str(e).strip('[]')==f):
                fp[e]=1
    return fp
l=['0000FB63AC1F3681EC880468', '1007CEA7AC1F3681EC880468','1007FB63AC1F3681EC880468']


#for d in l:
#    dataSet.drop((dataSet['ECP']==d).index)
#print("FirstSize",dataSet.size)
#dataSet.drop(dataSet[(dataSet['ECP']=='1007FB63AC1F3681EC880468')].index, inplace=True)
#print("SecondSize",dataSet.size)      


SecondSize 832


### Génération du DataSet
Nous allons générer l'ensemble des colonnes necessaires pour l'apprentissage et la classification

In [74]:
from scipy.stats import variation 

def generateDataSet(data, groupedBy):
    grouped_df = data.groupby(groupedBy) #On groupe les données par ..."Etiquetes"
    
    dataSet = pd.DataFrame({}) #Création d'une nouvelle dataFrame
    
    #Création des variables intermédiaire pour la récupération des champs
    ecp=pd.Series(grouped_df.ECP.unique().index,name="ECP")
    rc= pd.Series(grouped_df.ECP.count(),name="readcount")
    minRSSI= pd.Series(grouped_df.RSSI.min(),name="minRssi")
    maxRSSI= pd.Series(grouped_df.RSSI.max(),name="maxRssi")
    mean=pd.Series(grouped_df.RSSI.mean(),name="meanRssi")
    startTime=pd.Series(grouped_df.TimesTamp.min(),name="startTime")
    endTime=pd.Series(grouped_df.TimesTamp.max(),name="endTime")
    diff=pd.Series(grouped_df.RSSI.diff,name="difference")
    
    #Mise à jour des champs de la dataFrame
    dataSet['ECP']=ecp.values
    dataSet['RC']=rc.values
    dataSet['MIN']=minRSSI.values
    dataSet['MAX']=maxRSSI.values
    dataSet['MEAN']=mean.values
    dataSet['START']=startTime.values
    dataSet['END']=endTime.values
    dataSet['DURATION']=endTime.values-startTime.values
    cv={}
    a1={}
    a2={}
    a3={}
    a4={}
    for key, item in grouped_df:
        cv[key]=variation(grouped_df['RSSI'].get_group(key).values,axis=0)
        a1[key]=0
        a2[key]=0
        a3[key]=0
        a4[key]=0
        for a in grouped_df['Antenna'].get_group(key).values:
            if(a==1): a1[key]+=1
            if(a==2): a2[key]+=1
            if(a==3): a3[key]+=1
            if(a==4): a4[key]+=1
                
    dataSet['CV']=pd.Series(cv).values
    dataSet['A1']=pd.Series(a1).values
    dataSet['A2']=pd.Series(a2).values
    dataSet['A3']=pd.Series(a3).values
    dataSet['A4']=pd.Series(a4).values
    
    #A commenter lors du 
    #fp=setFalsePositive(fp_list, dataSet)  
    #dataSet['FP']=pd.Series(fp).values  
    return dataSet

dataSet=generateDataSet(data,'ECP')
dataSet.head(10)
print(dataSet.size)

936


In [77]:
#dataSet=dataSet.drop([3],axis=0) #Suppression de la 1ere ligne

#dataSet[dataSet.ECP!="0000FB63AC1F3681EC880468"]
dataSet.head(5)

ECP   RC   MIN   MAX       MEAN             START  \
0  0000FB63AC1F3681EC880468  225 -69.0 -52.5 -61.195556  1581079347416077   
1  1007CEA7AC1F3681EAC5106D   18 -65.0 -56.5 -60.000000  1581079349961880   
2  1007CEA7AC1F3681EC880468   26 -69.0 -60.0 -63.038462  1581079349403036   
3  1007FB63AC1F3681EC880468  352 -69.5 -50.0 -60.995739  1581079347403888   
4  2005FB63AC1F3681AB1004E3   18 -65.0 -54.5 -60.361111  1581079350842864   

                END  DURATION        CV  A1  A2   A3  A4  
0  1581079423874121  76458044 -0.053374  75  41   52  57  
1  1581079422372381  72410501 -0.040445   1  11    6   0  
2  1581079423940056  74537020 -0.034708   3   7    7   9  
3  1581079423959257  76555369 -0.056087  86  80  100  86  
4  1581079422385526  71542662 -0.045270   0  10    8   0

### Export Data
Nous allons sauvegarder les donées dans un fichier .csv

In [75]:
import time
def exportData(data, path):
    data.to_csv(path, index = None, header=False)
    

In [76]:
tm = time.strftime("%Y-%b-%d-%H_%M_%S")
path='../2-TransformedData/p30_'+tm+'.csv'
exportData(dataSet, path)

### Split Data
Nous allons découper les données en deux, 80% des trainSets et 20% des tesSets

In [79]:
def splitDataLE20(data):
    dataSet_np = np.random.rand(len(data)) <0.8
    trainSet=data[dataSet_np]
    testSet=data[~dataSet_np]
    return trainSet, testSet

from sklearn.model_selection import train_test_split
def sliptDataE20(data):
    X = data.iloc[:, :-1].values
    y = data['FP'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    trainSet=pd.DataFrame(X_train)
    trainSet[13]=y_train
    
    testSet = pd.DataFrame(X_test)
    testSet[13] = y_test
    return trainSet, testSet

In [80]:
tm = time.strftime("%Y-%b-%d-%H_%M_%S")
def importData2(path):
    #columnNames=["ECP","RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4","FP"]
    columnNames=["RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4","FP"]
    delimit=','
    data = pd.read_csv(path, names=columnNames, sep=delimit)
    #data = data.drop(columns="ECP")
    return data
you_data=importData2('../2-TransformedData/dataSets_you.csv')

trainSet, testSet = sliptDataE20(you_data)
#trainSet, testSet = sliptDataE20(dataSet)
#trainSet, testSet = splitDataLE20(dataSet)

pathTrain='../3-DataSet/trainSet_you_'+tm+'.csv'
exportData(dataSet, pathTrain)

pathTest='../3-DataSet/testSet_you_'+tm+'.csv'
exportData(dataSet, pathTest)